# U.S. Presidents and Macroeconomic Indicators

This notebook combines presidential term dates with FRED macroeconomic indicators.

## What this notebook does
- Loads presidential term data from `presidents.csv`
- Pulls annualized macro data from FRED
- Computes start/end term values, percent/absolute changes, and annualized changes
- Produces ranked comparison tables by metric
- Exports styled ranked tables to PNG files

## Repeatable run order
Run code cells in order from top to bottom. For a clean rerun, use **Restart Kernel and Run All**.

## Notes
- Some indicators begin in later years, so earlier presidents can have missing values.
- Exported images are written to the `exports/` folder.

In [1]:
from pathlib import Path

import pandas as pd
import numpy as np
import pandas_datareader as pdr

TOP_N = 5
EXPORT_DIR = Path("exports")
TABLE_CONVERSION = "matplotlib"

## 1) Load presidential terms

In [2]:
p_df = pd.read_csv('presidents.csv')
p_df["Starting Year"] = p_df["Term"].astype(str).str.extract(r"(\d{4})")[0].astype("Int64")
p_df["Ending Year"] = p_df["Term"].astype(str).str.extract(r"(\d{4})$")[0].astype("Int64")
p_df.insert(
    p_df.columns.get_loc("Ending Year") + 1,
    "Term Length",
    (p_df["Ending Year"] - p_df["Starting Year"]).astype("Int64"),
)
p_df

,No.,Name,Birthplace,Birthday,Life,Height,Children,Religion,Higher Education,Occupation,...,Vice President,Previous Office,Economy,Foreign Affairs,Military Activity,Other Events,Legacy,Starting Year,Ending Year,Term Length
0,1,George Washington,"Pope's Creek, VA",22-Feb,1732-1799,1.88,0,Episcopalian,NaN,"Plantation Owner, Soldier",...,John Adams,Commander-in-Chief,"[' Hamilton established BUS', '1792 Coinage Ac...",['1793 Neutrality in the France-Britain confli...,['1794 Whiskey Rebellion'],"['1791 Bill of Rights', '1792 Post Office foun...",He is universally regarded as one of the great...,1789,1797,8
1,2,John Adams,"Braintree, MA",30-Oct,1735-1826,1.70,5,Unitarian,Harvard,"Lawyer, Farmer",...,Thomas Jefferson,1st Vice President of USA,['1798 Progressive land value tax of up to 1% ...,['1797 the XYZ Affair: a bribe of French agent...,['1798–1800 The Quasi war. Undeclared naval wa...,['1798 Alien & Sedition Act to silence critics...,One of the most experienced men ever to become...,1797,1801,4
2,3,Thomas Jefferson,"Goochland County, VA",13-Apr,1743-1826,1.89,6,unaffiliated Christian,College of William and Mary,"Inventor,Lawyer, Architect",...,"Aaron Burr, George Clinton",2nd Vice President of USA,['1807 Embargo Act forbidding foreign trade in...,['1805 Peace Treaty with Tripoli. Piracy stopp...,['1801-05 Naval operation against Tripoli and ...,"['1803 The Louisiana purchase', '1804 12th Ame...",Probably the most intelligent man ever to occ...,1801,1809,8
3,4,James Madison,"Port Conway, VA",16-Mar,1751-1836,1.63,0,Episcopalian,Princeton,"Plantation Owner, Lawyer",...,"George Clinton, Elbridge Gerry",Secretary of State,[' The first U.S. protective tariff was impose...,['1814 The Treaty of Ghent ends the War of 1812'],['1811 Tippecanoe battle (Harrison vs. Chief T...,['1811 Cumberland Road construction starts (fi...,His leadership in the War of 1812 was particul...,1809,1817,8
4,5,James Monroe,"Monroe Hall, VA",28-Apr,1758-1831,1.83,2,Episcopalian,College of William and Mary,"Plantation Owner, Lawyer",...,Daniel Tompkins,Secretary of War,['1819 Panic of 1819 (too much land speculatio...,"['1823 Monroe Doctrine', '1818 49th parallel s...",['1817 1st Seminole war against Seminole India...,"['1819 Florida ceded to US', ""1820 Missouri Co...",His presidency contributed to national defense...,1817,1825,8
5,6,John Quincy Adams,"Braintree, MA",11-Jul,1767-1848,1.70,4,Unitarian,Harvard,"Lawyer, Diplomat",...,John Calhoun,Secretary of State,"[' ""Internal improvements"" program (roads, por...",['Unsuccessful attempt to purchase Texas from ...,['None'],"[' Accused for ""corrupt bargain"" to obtain Cla...","He had been an excellent Secretary of State, m...",1825,1829,4
6,7,Andrew Jackson,"Waxhaw, NC",15-Mar,1767-1845,1.85,0,Presbyterian,NaN,"Soldier, Lawyer",...,"John Calhoun, Martin van Buren",Military Governor of Florida,['1832 The Bank War. Veto for rechartering of ...,[' Texas wins independence'],['1836 Alamo. 6000 Mexicans defeat 190 America...,"['1830 Indian Removal Act', ""1832 South Caroli...",Historians see in him both the best and the wo...,1829,1837,8
7,8,Martin van Buren,"Kinderhook, NY",5-Dec,1782-1862,1.68,4,Dutch Reformed,NaN,Lawyer,...,Richard Johnson,8th Vice President of USA,['1837 The Panic of 1837. Financial crisis & d...,[' Recognition of Republic of Texas; annex avo...,['1838 2nd Seminole war against Seminole India...,"['1838 ""The Trail of Tears"". Indians’ relocati...","An able man, but always regarded more as a shr...",1837,1841,4
8,9,William H. Harrison,"Charles City County, VA",9-Feb,1773-1841,1.73,1,Episcopalian,Hampden-Sydney College,Soldier,...,John Tyler,Minister to Colombia,['None'],['None'],['None'],['1841 Delivered the longest inaugural address...,none,1841,1841,0
9,10,John Tyler,"Charles City County, VA",29-Mar,1790-1862,1.83,1,Episcopalian,College of William and Mary,Lawyer,...,none,10th Vice President of USA,['Economic crisis initiated by the Panic of 18...,['1842 Webster–Ashburton Treaty settles border.

## 2) Pull and annualize macroeconomic indicators

Each series is pulled from FRED and then averaged to yearly values for easier term-level comparisons.

In [3]:
def load_annual_series(series_id: str, start: str, value_name: str | None = None) -> pd.DataFrame:
    series = pdr.DataReader(series_id, "fred", start=start)
    if value_name and value_name != series_id:
        series = series.rename(columns={series_id: value_name})
    series["YEAR"] = series.index.year
    return series.groupby("YEAR").mean()


gdp = load_annual_series("GDP", "1947-01-01")
unemployment = load_annual_series("UNRATE", "1948-01-01")
population = load_annual_series("POP", "1952-01-01")
gini = load_annual_series("SIPOVGINIUSA", "1967-01-01", value_name="GINI")
labor_force = load_annual_series("CIVPART", "1948-01-01")
interest_rate = load_annual_series("DFF", "1954-01-01")
surplus_deficit = load_annual_series("FYFSD", "1947-01-01")
cpi = load_annual_series("CPIAUCSL", "1947-01-01", value_name="CPI")
pce = load_annual_series("PCEPI", "1947-01-01", value_name="PCE")

macro_frames = [
    gdp,
    unemployment,
    population,
    gini,
    labor_force,
    interest_rate,
    surplus_deficit,
    cpi,
    pce,
]

## 3) Merge macro data to presidential terms and compute changes

This step matches each indicator to the term start/end year and calculates absolute and percent changes.

In [4]:
merged_data = p_df.copy()
for frame in macro_frames:
    merged_data = merged_data.merge(frame, left_on="Starting Year", right_index=True, how="left")

merged_data = merged_data.rename(
    columns={
        "FYFSD": "Starting Surplus/Deficit",
        "GINI": "Starting Gini",
        "DFF": "Starting Interest Rate",
        "GDP": "Starting GDP",
        "UNRATE": "Starting Unemployment",
        "POP": "Starting Population",
        "CIVPART": "Starting Labor Force",
        "CPI": "Starting CPI",
        "PCE": "Starting PCE",
    }
)

series_lookup = {
    "Surplus/Deficit": (surplus_deficit, "FYFSD"),
    "Gini": (gini, "GINI"),
    "Interest Rate": (interest_rate, "DFF"),
    "GDP": (gdp, "GDP"),
    "Unemployment": (unemployment, "UNRATE"),
    "Population": (population, "POP"),
    "Labor Force": (labor_force, "CIVPART"),
    "CPI": (cpi, "CPI"),
    "PCE": (pce, "PCE"),
}

term_length = merged_data["Term Length"].replace({0: np.nan})

for metric, (frame, source_col) in series_lookup.items():
    start_col = f"Starting {metric}"
    end_col = f"Ending {metric}"
    delta_col = f"Change in {metric} Δ"
    pct_col = f"Change in {metric} %"
    annualized_col = f"Annualized Change in {metric} %"

    merged_data[end_col] = merged_data["Ending Year"].map(frame[source_col])
    merged_data[delta_col] = merged_data[end_col] - merged_data[start_col]
    merged_data[pct_col] = (merged_data[delta_col] / merged_data[start_col]) * 100
    merged_data[annualized_col] = np.where(
        (merged_data[start_col] > 0) & (merged_data[end_col] > 0) & (term_length > 0),
        ((merged_data[end_col] / merged_data[start_col]) ** (1 / term_length) - 1) * 100,
        np.nan,
    )

merged_data["Surplus/Deficit Ranking"] = merged_data["Change in Surplus/Deficit Δ"].rank(ascending=False)

merged_data["GDP per Capita Starting"] = (merged_data["Starting GDP"] / merged_data["Starting Population"]) * 1_000_000
merged_data["GDP per Capita Ending"] = (merged_data["Ending GDP"] / merged_data["Ending Population"]) * 1_000_000
merged_data["Change in GDP per Capita Δ"] = merged_data["GDP per Capita Ending"] - merged_data["GDP per Capita Starting"]
merged_data["Change in GDP per Capita %"] = (
    merged_data["Change in GDP per Capita Δ"] / merged_data["GDP per Capita Starting"]
) * 100
merged_data["Annualized Change in GDP per Capita %"] = np.where(
    (merged_data["GDP per Capita Starting"] > 0) & (merged_data["GDP per Capita Ending"] > 0) & (term_length > 0),
    ((merged_data["GDP per Capita Ending"] / merged_data["GDP per Capita Starting"]) ** (1 / term_length) - 1) * 100,
    np.nan,
)

## 4) Build ranked comparison tables

In [5]:
gdp_ranked = merged_data.sort_values("Change in GDP %", ascending=False).dropna(subset=["Change in GDP %"])
gdp_ranked = gdp_ranked[
    ["Name", "Term", "Term Length", "Starting GDP", "Ending GDP", "Change in GDP Δ", "Change in GDP %", "Annualized Change in GDP %"]
]

gdp_per_capita_ranked = merged_data.sort_values("Change in GDP per Capita %", ascending=False).dropna(
    subset=["Change in GDP per Capita %"]
)
gdp_per_capita_ranked = gdp_per_capita_ranked[
    [
        "Name",
        "Term",
        "Term Length",
        "GDP per Capita Starting",
        "GDP per Capita Ending",
        "Change in GDP per Capita Δ",
        "Change in GDP per Capita %",
        "Annualized Change in GDP per Capita %",
    ]
]

gini_ranked = merged_data.sort_values("Change in Gini %", ascending=True).dropna(subset=["Change in Gini %"])
gini_ranked = gini_ranked[
    ["Name", "Term", "Term Length", "Starting Gini", "Ending Gini", "Change in Gini Δ", "Change in Gini %", "Annualized Change in Gini %"]
]

unemployment_ranked = merged_data.sort_values("Change in Unemployment %", ascending=True).dropna(
    subset=["Change in Unemployment %"]
)
unemployment_ranked = unemployment_ranked[
    [
        "Name",
        "Term",
        "Term Length",
        "Starting Unemployment",
        "Ending Unemployment",
        "Change in Unemployment Δ",
        "Change in Unemployment %",
        "Annualized Change in Unemployment %",
    ]
]

labor_force_ranked = merged_data.sort_values("Change in Labor Force %", ascending=False).dropna(
    subset=["Change in Labor Force %"]
)
labor_force_ranked = labor_force_ranked[
    [
        "Name",
        "Term",
        "Term Length",
        "Starting Labor Force",
        "Ending Labor Force",
        "Change in Labor Force Δ",
        "Change in Labor Force %",
        "Annualized Change in Labor Force %",
    ]
]

cpi_ranked = merged_data.sort_values("Change in CPI %", ascending=True).dropna(subset=["Change in CPI %"])
cpi_ranked = cpi_ranked[
    ["Name", "Term", "Term Length", "Starting CPI", "Ending CPI", "Change in CPI Δ", "Change in CPI %", "Annualized Change in CPI %"]
]

pce_ranked = merged_data.sort_values("Change in PCE %", ascending=True).dropna(subset=["Change in PCE %"])
pce_ranked = pce_ranked[
    ["Name", "Term", "Term Length", "Starting PCE", "Ending PCE", "Change in PCE Δ", "Change in PCE %", "Annualized Change in PCE %"]
]

## 5) Review outputs

In [6]:
top_n = TOP_N

rank_tables = {
    "GDP": gdp_ranked,
    "GDP per Capita": gdp_per_capita_ranked,
    "Gini (Lower Better)": gini_ranked,
    "Unemployment (Lower Better)": unemployment_ranked,
    "Labor Force": labor_force_ranked,
    "CPI Inflation (Lower Better)": cpi_ranked,
    "PCE Inflation (Lower Better)": pce_ranked,
}

top5_summary = pd.concat(
    [
        table[["Name", "Term"]].head(top_n).reset_index(drop=True).rename(
            columns={"Name": f"{label} - Name", "Term": f"{label} - Term"}
        )
        for label, table in rank_tables.items()
    ],
    axis=1,
)

top5_summary

,GDP - Name,GDP - Term,GDP per Capita - Name,GDP per Capita - Term,Gini (Lower Better) - Name,Gini (Lower Better) - Term,Unemployment (Lower Better) - Name,Unemployment (Lower Better) - Term,Labor Force - Name,Labor Force - Term,CPI Inflation (Lower Better) - Name,CPI Inflation (Lower Better) - Term,PCE Inflation (Lower Better) - Name,PCE Inflation (Lower Better) - Term
0,Ronald Reagan,1981-1989,Ronald Reagan,1981-1989,Donald Trump,2017-2021,Barack Obama,2009-2017,Ronald Reagan,1981-1989,John F. Kennedy,1961-1963,John F. Kennedy,1961-1963
1,Lyndon Johnson,1963-1969,Lyndon Johnson,1963-1969,Richard Nixon,1969-1974,Lyndon Johnson,1963-1969,Jimmy Carter,1977-1981,Donald Trump,2017-2021,Donald Trump,2017-2021
2,Bill Clinton,1993-2001,Jimmy Carter,1977-1981,Jimmy Carter,1977-1981,Bill Clinton,1993-2001,Lyndon Johnson,1963-1969,Dwight Eisenhower,1953-1961,Barack Obama,2009-2017
3,Jimmy Carter,1977-1981,Richard Nixon,1969-1974,Gerald R. Ford,1974-1977,Ronald Reagan,1981-1989,Richard Nixon,1969-1974,Barack Obama,2009-2017,George H. W. Bush,1989-1993
4,Richard Nixon,1969-1974,Bill Clinton,1993-2001,George W. Bush,2001-2009,Joe Biden,2021-2025,Gerald R. Ford,1974-1977,George H. W. Bush,1989-1993,Bill Clinton,1993-2001


In [8]:
import dataframe_image as dfi
from pandas.io.formats.style import Styler

def ensure_styler(table, formatter: dict[str, str], precision: int = 2):
    base_table = table.data if isinstance(table, Styler) else table
    return base_table.style.format(precision=precision, formatter=formatter)

def export_ranked_tables(ranked_tables: dict[str, Styler], export_dir: Path, table_conversion: str) -> list[str]:
    export_dir.mkdir(parents=True, exist_ok=True)
    exported_files: list[str] = []

    for table_name, table in ranked_tables.items():
        safe_name = table_name.replace(" ", "_").replace("/", "_")
        output_path = export_dir / f"{safe_name}.png"
        dfi.export(table, str(output_path), table_conversion=table_conversion)
        exported_files.append(str(output_path))

    return exported_files

formatters = {
    "GDP": {"Change in GDP %": "{:.2f}%", "Annualized Change in GDP %": "{:.2f}%"},
    "GDP per Capita": {
        "Change in GDP per Capita %": "{:.2f}%",
        "Annualized Change in GDP per Capita %": "{:.2f}%",
    },
    "Gini (Lower Better)": {"Change in Gini %": "{:.2f}%", "Annualized Change in Gini %": "{:.2f}%"},
    "Unemployment (Lower Better)": {
        "Change in Unemployment %": "{:.2f}%",
        "Annualized Change in Unemployment %": "{:.2f}%",
    },
    "Labor Force": {
        "Change in Labor Force %": "{:.2f}%",
        "Annualized Change in Labor Force %": "{:.2f}%",
    },
    "CPI Inflation (Lower Better)": {"Change in CPI %": "{:.2f}%", "Annualized Change in CPI %": "{:.2f}%"},
    "PCE Inflation (Lower Better)": {"Change in PCE %": "{:.2f}%", "Annualized Change in PCE %": "{:.2f}%"},
}

base_ranked_tables = {
    "GDP": gdp_ranked,
    "GDP per Capita": gdp_per_capita_ranked,
    "Gini (Lower Better)": gini_ranked,
    "Unemployment (Lower Better)": unemployment_ranked,
    "Labor Force": labor_force_ranked,
    "CPI Inflation (Lower Better)": cpi_ranked,
    "PCE Inflation (Lower Better)": pce_ranked,
}

ranked_tables = {
    table_name: ensure_styler(base_ranked_tables[table_name], formatters[table_name])
    for table_name in base_ranked_tables
}

gdp_ranked = ranked_tables["GDP"]
gdp_per_capita_ranked = ranked_tables["GDP per Capita"]
gini_ranked = ranked_tables["Gini (Lower Better)"]
unemployment_ranked = ranked_tables["Unemployment (Lower Better)"]
labor_force_ranked = ranked_tables["Labor Force"]
cpi_ranked = ranked_tables["CPI Inflation (Lower Better)"]
pce_ranked = ranked_tables["PCE Inflation (Lower Better)"]

exported_files = export_ranked_tables(
    ranked_tables=ranked_tables,
    export_dir=EXPORT_DIR,
    table_conversion=TABLE_CONVERSION,
)

exported_files

['exports\\GDP.png',
 'exports\\GDP_per_Capita.png',
 'exports\\Gini_(Lower_Better).png',
 'exports\\Unemployment_(Lower_Better).png',
 'exports\\Labor_Force.png',
 'exports\\CPI_Inflation_(Lower_Better).png',
 'exports\\PCE_Inflation_(Lower_Better).png']

In [ ]:
gdp_per_capita_ranked

## 6) Export artifacts

Running Cell 13 exports one PNG per ranked table to `exports/`.
This step is safe to run repeatedly and will overwrite files with the same names.